In [1]:
import json
import pandas as pd
import numpy as np

In [12]:
def main_all(delta_card, sum_player):
    for idx, pos in enumerate(position_list):
        df_new = main_position(pos, delta_card, sum_player)
        if idx > 0:
            frames = [df, df_new]
            df = pd.concat(frames, ignore_index=True)
        else:
            df = df_new
    df = df.sort_values(by='Value', ascending=False)
    return df

def main_position(position, delta_card, sum_player):
    if delta_card:
        df1 = dataframe_card_delta(position, version_list[0])
        df2 = dataframe_card_delta(position, version_list[1])
    else:
        df1 = dataframe_base(time_list[1], position, version_list[0])
        df2 = dataframe_base(time_list[1], position, version_list[1])
    
    frames = [df1, df2]
    df = pd.concat(frames, ignore_index=True)
    
    if sum_player:
        df = dataframe_player_sum(df)
    
    df["Value"] = value(df.Games, df.Price)
    df = df.sort_values(by='Value', ascending=False)
    return df

def dataframe_card_delta(position, version):
    df0 = dataframe_base(time_list[0], position, version)
    df1 = dataframe_base(time_list[1], position, version)
    
    dn = df1.Games.to_numpy()
    J = np.zeros(len(df1.index))
    i = -1
    for i in range(len(df1.index)):
        namei = df1.iat[i, 0]
        Ri = df1.iat[i, 1]
        Ni = df1.iat[i, 2]
        Pi = df1.iat[i, 3]
        for j in range(len(df0.index)):
            namej = df0.iat[j, 0]
            Rj = df0.iat[j, 1]
            Nj = df0.iat[j, 2]
            Pj = df0.iat[j, 3]
#             print("%s %s %s %s - %s %s %s %s " % (namei, Ri, Ni, Pi, namej, Rj, Nj, Pj))
            if J[j] == 1:
                continue
            if namei == namej and Ri == Rj and Nj <= Ni:
#                 print("%s %s %s %s - %s %s %s %s " % (namei, Ri, Ni, Pi, namej, Rj, Nj, Pj))
                dn[i] -= Nj
                J[j] = 1
                break
    df1.Games = dn.astype(int)
    return df1

def dataframe_base(time, position, version):
    df = dataframe_import(time, position, version)
    
    df = df.loc[:, ["Name", "Rating", "Games", "PS4"]]
    df.rename(columns={'PS4':'Price'}, inplace=True)
    df.Name = df.Name.str.replace(r"\(.*\)","")
    df.Rating = pd.to_numeric(df.Rating)
    df.Games = pd.to_numeric(df.Games.str.replace(",",""))
    df.Price = (df.Price.replace(r'[KM]+$', '', regex=True).astype(float) * \
                    df.Price.str.extract(r'[\d\.]+([KM]+)', expand=False)
                    .fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int))
    df.Price = df.Price.astype("int64")
    return df

def dataframe_import(time, position, version):
    filename = "../scraped_data/%s_%s_%s.json" % (time, position, version)
    with open(filename, encoding='utf-8') as json_data:
        d = json.load(json_data)
        json_data.close()
    df = pd.DataFrame(d)
    return df

def dataframe_player_sum(df):
    Nsum = np.zeros(len(df.index))
    Pweight = np.zeros(len(df.index))
    Pmin = np.zeros(len(df.index))
    for i in range(len(df.index)):
        namei = df.iat[i, 0]
        Ri = df.iat[i, 1]
        Ni = df.iat[i, 2]
        Pi = df.iat[i, 3]
        Nsum[i] = Ni
        Pweight[i] = Pi * Ni
        Pmin[i] = Pi
        for j in range(i + 1,len(df.index)):
            namej = df.iat[j, 0]
            Rj = df.iat[j, 1]
            Nj = df.iat[j, 2]
            Pj = df.iat[j, 3]
            if namej == namei:
#                 print("%s %s %s %s - %s %s %s %s " % (namei, Ri, Ni, Pi, namej, Rj, Nj, Pj))
                Nsum[i] += Nj
                Pweight[i] += Nj * Pj
                if Pj < Pmin[i] or Pmin[i] == 0:
                    Pmin[i] = Pj
                df.iat[j, 2] = 0
                df.iat[j, 3] = 0
        if Nsum[i] > 0:
            Pweight[i] = Pweight[i] / Nsum[i]
    df.Games = Nsum
    df.Games = df.Games.astype("int64")
    df.Price = Pweight
    df.Price = df.Price.astype("int64")
    df["Price_min"] = Pmin
    df.Price_min = df.Price_min.astype("int64")
#     df["Value"] = value(df.Games, df.Price)
#     df = df.sort_values(by='Value', ascending=False)
    df = df.reindex(columns=["Name", "Games", "Price", "Price_min", "Value"])
    return df

def value(games, price):
    P = np.array(price.to_list())
    N = np.array(games.to_list())
    V = (A + (1 - A) / (1 + np.exp(-a * (P - b)))) * (c * N + d)
    return V


In [3]:
sat = 100000  # price at exponent = satx
satx = 2      # exponent at P = sat
zero = 0.93   # pricevalue at P = 0, price off = 1
zerox = -1    # exponent at P = 0
b = sat/(satx - zerox)
a = satx / (sat - b)
A = (zero * (1 + np.exp(a * b)) -1) / np.exp(a * b)
c = 1         # games off = 0
d = 0         # games off = 1

version_list = ["all_nif", "all_specials"]
position_list = ["GK", "RB,RWB", "LB,LWB", "CB", "CDM,CM,CAM", "RM,RW,RF", "LM,LW,LF", "CF,ST"]
time_list = ["2019121307", "2020010808"]

In [14]:
df = main_all(True, True)
# df = main_position(position_list[2], True, True)
df.Value = df.Value.div(df.iat[0, 4])
df.head(60)

,Name,Games,Price,Price_min,Value
420,Wissam Ben Yedder,6810585,336154,3400,1.000000
0,Marc-André ter Stegen,3554872,51888,40250,0.503760
240,Frenkie de Jong,2941808,109880,11500,0.428170
241,Allan,2796729,129697,9000,0.408580
180,Éder Militão,2713415,338592,1400,0.398412
300,Ousmane Dembélé,2703903,334352,7800,0.397015
301,Riyad Mahrez,2426493,80072,4500,0.349550
360,Coutinho,2276093,102976,16750,0.330678
242,Fabinho,2329901,28739,9900,0.324598
1,Hugo Lloris,1974368,70274,26000,0.283010


In [8]:
import matplotlib.pyplot as plt
# print("x = 2 at P = %.0f" % (2/a+b))
# print("x = %.0f" % (a*(sat - b)))
# print(df.loc[3,"Value"]-df.loc[2,"Value"])

x = np.linspace(0,1e6)
y = (A + (1 - A) / (1 + np.exp(-a * (x - b))))

plt.plot(x,y)
plt.axis([0, 2e5, 0, 1])
plt.show()

<Figure size 640x480 with 1 Axes>